In [16]:
#Scrape the NASA Mars News Site and collect the latest News Title and Paragragh Text
#Scrape featured image from JPL Featured Space Image
#Obtain Mars size vs. Earth diagram and alt/text caption
#Scrape Mars planet facts and information
#Obtain latest Mars weather information (@marswxreport, Twitter)
#Obtain full size image url links for all of Mars' Hemispheres

#Imports
from splinter import Browser #https://splinter.readthedocs.io/en/latest/
from bs4 import BeautifulSoup #https://www.crummy.com/software/BeautifulSoup/bs4/doc/

import pandas as pd
import time
import datetime
import requests
import json

In [17]:
#Set path for splinter (chromedriver)
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
time.sleep(1.5)

In [21]:
#Scrape the NASA Mars News Site and collect the latest News Title and Paragragh Text

#url to scrape
url = "https://mars.nasa.gov/news/"
browser.visit(url)
#set url in splinter/chromedriver
html = browser.html
soup = BeautifulSoup(html, "html.parser")
time.sleep(1)
news_title = soup.find('div',class_='content_title').get_text()
time.sleep(1.5)
news_p = soup.find('div',class_='article_teaser_body').text
print(news_title)
print(news_p)


NASA Invests in Visionary Technology 
NASA is investing in technology concepts, including several from JPL, that may one day be used for future space exploration missions.


In [7]:
#Scrape featured image from JPL Featured Space Image
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
time.sleep(3)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(1.5)
browser.click_link_by_partial_text('more info')

#Pass html from splinter/chromedriver to be parsed by beautiful soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

#Obtain relative image url from parsed html with beautiful soup
img_url_part = soup.find('figure', class_='lede').find('img')['src']
print("Relative url of image:")
print(img_url_part)
base_url =" https://www.jpl.nasa.gov"
#Concatenate base_url and img_url_part
print("Reconstructed full url of full size image:")
print(base_url + img_url_part)

Relative url of image:
/spaceimages/images/largesize/PIA18358_hires.jpg
Reconstructed full url of full size image:
 https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18358_hires.jpg


In [8]:
#Scrape Mars planet facts and information
url = 'http://space-facts.com/mars/'
browser.visit(url)
time.sleep(3)

#Pass html from splinter/chromedriver to be parsed by beautiful soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

#Pass html from beautiful soup to the pd.read_html method:
mars_facts_df = pd.read_html(html)[0]
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_html.html
mars_facts_df .columns=['description', 'value']
mars_facts_df.set_index('description', inplace=True)

#Mars size vs. Earth diagram scrape:
diagram_text= soup.find('div',id="attachment_4837").find('img')['alt']
diagram_url = soup.find('div',id="attachment_4837").find('img')['src']

#Display output of table unformatted:
mars_facts_df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>

In [9]:
#Url of Mars size vs. Earth diagram and alt text/caption:
print(diagram_url)
print(diagram_text)

https://space-facts.com/wp-content/uploads/mars-size.png
Mars size compared to Earth


In [10]:
#Examine dataframe conversion of table:
mars_facts_df.head(15)

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [11]:
#Latest mars weather information (@marswxreport, Twitter)
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
time.sleep(1)

#Pass html from splinter/chromedriver to be parsed by beautiful soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

mars_weather_tweet_object = soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})
mars_weather_tweet_enc_tags= mars_weather_tweet_object.find('p', 'tweet-text')
current_mars_weather = mars_weather_tweet_enc_tags.get_text()
print(current_mars_weather)

Sol 2004 (March 27, 2018), Sunny, high -8C/17F, low -74C/-101F, pressure at 7.16 hPa, daylight 05:31-17:23


In [12]:
#Mars Hemispheres title and image url
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
time.sleep(1)

#Pass html from splinter/chromedriver to be parsed by beautiful soup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

#Base url for full image link reconstruction
url_base = "https://astrogeology.usgs.gov"

#List declaration
hemisphere_url_images_complete = []
url_list_incomplete = []


In [13]:
#Find all div elements of the item class for url list construction
result = soup.find_all('div', class_="item")
#Incomplete url list construction:
for item in result:
    link = item.find('a')['href']
    print(item.find('a')['href'])
    url_list_incomplete.append(link)

/search/map/Mars/Viking/cerberus_enhanced
/search/map/Mars/Viking/schiaparelli_enhanced
/search/map/Mars/Viking/syrtis_major_enhanced
/search/map/Mars/Viking/valles_marineris_enhanced


In [14]:
for incomplete_url in url_list_incomplete:
    #reconstruct complete url with u
    url = url_base + incomplete_url
    #visit complete url with splinter/chromedriver   
    browser.visit(url)
    time.sleep(1.5)
    
    #Pass html from splinter/chromedriver to be parsed by beautiful soup
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    #Grab full image url and reconstruct with base url
    full_image_partial_url = soup.find('img', class_="wide-image")
    image = url_base + full_image_partial_url["src"]
    
    # Grab page title and remove "Enhanced" from string
    result2 = soup.find('h2', class_='title').text
    #title = result2.text
    title = result2.rsplit(' ', 1)[0]
    
    hemisphere_dict = {"Title": title, "Image URL": image}
    hemisphere_url_images_complete.append(hemisphere_dict)
    
    time.sleep(1.5)
    
#print(hemisphere_url_images_complete)
print(json.dumps(hemisphere_url_images_complete,indent=1))

[
 {
  "Title": "Cerberus Hemisphere",
  "Image URL": "https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg"
 },
 {
  "Title": "Schiaparelli Hemisphere",
  "Image URL": "https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg"
 },
 {
  "Title": "Syrtis Major Hemisphere",
  "Image URL": "https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg"
 },
 {
  "Title": "Valles Marineris Hemisphere",
  "Image URL": "https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg"
 }
]


In [15]:
#Close chrome window (splinter/chromedriver)
browser.quit()